# Used to help intrepet the output from the cpac outline

In [26]:
import os
import pickle
import numpy as np
from subprocess import call

import sys
sys.path.append('../src/features')

from subject import Subject

cur_dir = os.getcwd()
asd_diag_dir = os.path.dirname(cur_dir)
data_dir = asd_diag_dir + '/data/'
# cpac_output_dir = data_dir + 'cpac_output_t/'
cpac_output_dir = data_dir + 'cpac_output/output/'
subjects_dir = os.path.dirname(cur_dir) + '/data/ABIDEI_subjects/'
trs_save_file = save_dir = os.path.dirname(cur_dir) + '/data/dicts/ABIDEI_site_trs.json'

In [98]:
sub_number = 51471
subject_folder = cpac_output_dir + f'/pipeline_abide_cpac_run_freq-filter_nuisance/sub-00{sub_number}_ses-1/'

In [99]:
# Find all cc200 rois within output
# folder_queue = os.listdir(cpac_output_dir)
# folder_queue = [os.path.join(cpac_output_dir, f) for f in folder_queue]
folder_queue = os.listdir(subject_folder)
folder_queue = [os.path.join(subject_folder, f) for f in folder_queue]
cc_200_l = list()

while len(folder_queue) > 0:
    f = folder_queue.pop(0)
    if(os.path.exists(os.path.join(f, 'roi_stats.npz'))):
        cc_200_l.append(f)
    for d in os.listdir(f):
        if os.path.isdir(os.path.join(f, d)):
            folder_queue.append(os.path.join(f, d))

In [100]:
len(cc_200_l)

54

In [101]:
for d in cc_200_l:
    print(d[200:])
    print()

5_G-M_M-SDB_P-2_BP-B0.01-T0.1/_mask_DKT_space-MNI152NLin6_res-1x1x1_mask_file_..ndmg_atlases..label..Human..DKT_space-MNI152NLin6_res-1x1x1.nii.gz

5_G-M_M-SDB_P-2_BP-B0.01-T0.1/_mask_Juelich_space-MNI152NLin6_res-1x1x1_mask_file_..ndmg_atlases..label..Human..Juelich_space-MNI152NLin6_res-1x1x1.nii.gz

5_G-M_M-SDB_P-2_BP-B0.01-T0.1/_mask_CC400_mask_file_..cpac_templates..CC400.nii.gz

5_G-M_M-SDB_P-2_BP-B0.01-T0.1/_mask_Brodmann_space-MNI152NLin6_res-1x1x1_mask_file_..ndmg_atlases..label..Human..Brodmann_space-MNI152NLin6_res-1x1x1.nii.gz

5_G-M_M-SDB_P-2_BP-B0.01-T0.1/_mask_rois_3mm_mask_file_..cpac_templates..rois_3mm.nii.gz

5_G-M_M-SDB_P-2_BP-B0.01-T0.1/_mask_Slab907_space-MNI152NLin6_res-1x1x1_mask_file_..ndmg_atlases..label..Human..Slab907_space-MNI152NLin6_res-1x1x1.nii.gz

5_G-M_M-SDB_P-2_BP-B0.01-T0.1/_mask_Yeo-7-liberal_space-MNI152NLin6_res-1x1x1_mask_file_..ndmg_atlases..label..Human..Yeo-7-liberal_space-MNI152NLin6_res-1x1x1.nii.gz

5_G-M_M-SDB_P-2_BP-B0.01-T0.1/_mask_Scha

In [102]:
cc_200_l[1]

'/home/cparish/Capstone/asd_diagnosis_fmri/data/cpac_output/output//pipeline_abide_cpac_run_freq-filter_nuisance/sub-0051471_ses-1/roi_timeseries/_scan_rest_run-1/_selector_CSF-2mmE-M_aC-CSF+WM-2mm-DPC5_G-M_M-SDB_P-2_BP-B0.01-T0.1/_mask_Juelich_space-MNI152NLin6_res-1x1x1_mask_file_..ndmg_atlases..label..Human..Juelich_space-MNI152NLin6_res-1x1x1.nii.gz'

# Analyze ROI output from cpac 

In [178]:
# Load cpac rois
cpac_rois = {}
for roi_folder in cc_200_l:
    call(['unzip', os.path.join(roi_folder, 'roi_stats.npz'), '-d', roi_folder])
    # first row is nans and skip first 5
    roi = np.load(os.path.join(roi_folder, 'arr_0.npy'), allow_pickle=True)[5:] 
    cpac_rois[roi_folder] = roi
    

In [179]:
len(cpac_rois)

54

In [167]:
# Load ABIDEI preprocessed rois by loading subjects
def open_pickle(f):
    file = open(f,'rb')
    o = pickle.load(file)
    file.close()
    return o

def load_subjects_d(subject_folder):
    subjects_d = {}
    for f in os.listdir(subject_folder):
        s = open_pickle(os.path.join(subject_folder, f))
        subjects_d[s._sub_id] = s
    return subjects_d

In [202]:
subjects_d = load_subjects_d(subjects_dir)

In [191]:
s = subjects_d[51471]

In [183]:
cc_200_rois = {}
for folder, roi in cpac_rois.items():
    if roi.shape == (146, 200) and folder.find('CC200') != -1:
        cc_200_rois[folder] = roi

In [184]:
len(cc_200_rois)

2

In [192]:
s._data_dict.keys()

dict_keys(['filt_noglobal_roi_200_Cradd', 'cpac_filt_noglobal'])

In [173]:
goal_roi = np.array(s._data_dict['filt_noglobal_roi_200_Cradd'])
goal_ts = goal_roi[3]
print(goal_ts.shape)

(200,)


In [188]:
min_indexes = list()
for folder, roi in cc_200_rois.items():
    print(f'{folder}:{np.linalg.norm(roi-goal_roi)}')
    min_index = 0
    min_dist = 200
    for i, ts in enumerate(roi):
        dist = np.linalg.norm(ts-goal_ts)
        if dist < min_dist:
            min_index = i
            min_dist = dist
        if(ts == goal_ts).all():
            print('d')
    min_indexes.append(min_index)
#     if(roi[5:] == goal_roi).all():
#         print(roi.shape)

/home/cparish/Capstone/asd_diagnosis_fmri/data/cpac_output/output//pipeline_abide_cpac_run_freq-filter_nuisance/sub-0051471_ses-1/roi_timeseries/_scan_rest_run-1/_selector_CSF-2mmE-M_aC-CSF+WM-2mm-DPC5_G-M_M-SDB_P-2_BP-B0.01-T0.1/_mask_CC200_mask_file_..cpac_templates..CC200.nii.gz:1367.8197454035635
/home/cparish/Capstone/asd_diagnosis_fmri/data/cpac_output/output//pipeline_abide_cpac_run_freq-filter_nuisance/sub-0051471_ses-1/roi_timeseries/_scan_rest_run-1/_selector_CSF-2mmE-M_aC-CSF+WM-2mm-DPC5_M-SDB_P-2_BP-B0.01-T0.1/_mask_CC200_mask_file_..cpac_templates..CC200.nii.gz:1208.4897610950802


In [187]:
dists

[1367.8197454035635, 1208.4897610950802]

In [124]:
np.array(s._data_dict['filt_noglobal_roi_200_Cradd']).shape

(146, 200)

In [125]:
goal_ts

array([ 1.1277800e+00,  4.6984740e+00, -2.1352580e+00, -5.2501800e-01,
        1.3163840e+00, -5.9062520e+00, -3.3105240e+00,  1.0040340e+00,
       -1.2913460e+00,  4.5389000e-01, -7.3150870e+00,  0.0000000e+00,
       -3.8072650e+00,  1.2421070e+00,  1.3271420e+00, -4.3659560e+00,
       -2.1308890e+00,  2.1773000e-01,  2.3604780e+00, -1.0887290e+01,
       -1.0051570e+00, -4.7868600e-01,  2.8610070e+00, -8.1607860e+00,
        1.4903890e+00, -8.7014380e+00,  0.0000000e+00, -4.1830290e+00,
        3.2639690e+00,  0.0000000e+00, -8.3632550e+00,  0.0000000e+00,
        9.7243100e+00,  3.1085830e+00, -1.3633828e+01,  0.0000000e+00,
       -2.4167000e+00, -1.5633151e+01, -8.4798100e+00, -8.6998580e+00,
       -3.9630000e-03,  0.0000000e+00,  0.0000000e+00, -5.6726810e+00,
       -2.4597800e-01,  4.6972760e+00, -8.6655400e-01, -7.2814870e+00,
       -5.5952060e+00, -3.9764200e-01,  3.2370000e-02,  0.0000000e+00,
       -1.7280400e-01, -5.1317790e+00, -1.6518399e+01,  9.8068950e+00,
      

In [116]:
for f in cc_200_rois.keys():
    print(f.split('/')[-1])

_mask_CC200_mask_file_..cpac_templates..CC200.nii.gz
_mask_Schaefer200_space-MNI152NLin6_res-1x1x1_mask_file_..ndmg_atlases..label..Human..Schaefer200_space-MNI152NLin6_res-1x1x1.nii.gz
_mask_CC200_mask_file_..cpac_templates..CC200.nii.gz
_mask_Schaefer200_space-MNI152NLin6_res-1x1x1_mask_file_..ndmg_atlases..label..Human..Schaefer200_space-MNI152NLin6_res-1x1x1.nii.gz


# Analyzing with Subjects

In [213]:
# for filter no global
filt_noglobal = 'roi_timeseries/_scan_rest_run-1/_selector_CSF-2mmE-M_aC-CSF+WM-2mm-DPC5_M-SDB_P-2_BP-B0.01-T0.1/_mask_CC200_mask_file_..cpac_templates..CC200.nii.gz'
def get_roi(sub_folder):
    roi_folder = os.path.join(sub_folder,filt_noglobal)
    if os.path.exists(os.path.join(roi_folder, 'roi_stats.npz')):
        call(['unzip', os.path.join(roi_folder, 'roi_stats.npz'), '-d', roi_folder])
        # first row is nans and skip first 5
        roi = np.load(os.path.join(roi_folder, 'arr_0.npy'), allow_pickle=True)[5:] 
    else:
        print(f"Couldn't get rois for {sub_folder}")
        roi = None
    return roi

sub_51470_roi = get_roi(data_dir + '/cpac_output_single/output/pipeline_abide_cpac_run_freq-filter_nuisance/sub-0051470_ses-1')

In [218]:
# run in same pipeline twice
sub_51470_roi_2 = subjects_d[51470]._data_dict['cpac_filt_noglobal'] 
dist = np.linalg.norm(sub_51470_roi-sub_51470_roi_2)
dist

112.76785058445424

In [203]:
cpac_output_subs = list()
for sub, s in subjects_d.items():
    if 'cpac_filt_noglobal' in s._data_dict:
        cpac_output_subs.append(s)

In [209]:
for s in cpac_output_subs:
    cpac_roi = s._data_dict['cpac_filt_noglobal']
    preprocessed_roi = s._data_dict['filt_noglobal_roi_200_Cradd']
    dist = np.linalg.norm(cpac_roi-preprocessed_roi)
    print(dist)

1458.0333812867184
2023.6840385753046
4317.593523370455
1963.3674292851229
1553.5067259525335
1871.23161751424
1684.2098021386132
1312.022369114149
1566.688771480727
1391.5073333761243
1208.4897610950802
1610.333895102181
1765.3053892936673
3255.6904802495624
1905.3217028186111
2945.5869832416356
1540.7676056260175
1642.4417198939084
2454.6677947874455
1954.6868357942733
1787.251580462886
1887.3317144633447
2031.976079376318
1869.1076857643045
1771.2490815315884
1467.2397242208772
1492.6002576736055
3736.181723233348
1308.1924505911238
3781.54321597015
1390.1382577164409
1606.728975342744
1826.2135369607254
1413.1954807102659


In [212]:
for i, s in enumerate(cpac_output_subs):
    print(s._sub_id)
    # roi_1 = s._data_dict['cpac_filt_noglobal']
    roi_1 = s._data_dict['filt_noglobal_roi_200_Cradd']
    s2 = cpac_output_subs[i + 1]
    # roi_2 = s2._data_dict['cpac_filt_noglobal']
    roi_2 = s2._data_dict['filt_noglobal_roi_200_Cradd']
    dist =  dist = np.linalg.norm(roi_2-roi_1)
    print(dist)

51470
4748.805419186433
51474
5235.379898062245
51457
4854.722798395948
51463
4000.8804137316943
51458
4451.414270977289
51485
4658.924489583965
51491
3720.696440446671
51480
3897.3305731112027
51456
3992.4066242406866
51483
3327.386993369306
51471
3613.0555523404705
51468
3986.0420483638045
51460
2904.913129176263
51478
4018.8713529784172
51462
6152.507402665735
51477
5223.970736246386
51466
4047.095173276084
51492
5002.568687937156
51488
5485.062750517512
51479
5172.5993045592095
51465
5329.076289377555
51467
5735.189042648404
51489
5341.695206039386
51473
4692.186201096127
51486
4960.440567269033
51487
4787.7947869584805
51476
4403.646144629601
51464
3762.5894667202924
51493
5887.827799660593
51461
6230.688022159607
51484
5026.128456040572
51482
5061.864978276973
51481
4855.742576658661
51490


IndexError: list index out of range